<img src="Images/pyg2.png" alt="" width="150"/> <br><br>

# A Chemists Guide to [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/)
<br>

> ### Chapters:
>
>* [Installation](#installation)
>* [Loading Existing Datasets](#loading-existing-datasets)
>* [Creating Your Own Datasets](#creating-your-own-datasets)
>* [Training a Simple Model](#training-a-simple-model)
>


<a id='installation'></a>

## Installation
PyTorch Geometric builds on the popular machine learning framework PyTorch and extends it by adding machine learning functionalities for graphs. In addition to the machine learning packages, we need to install RDKit to work with molecules. To install the packages using anaconda, open a terminal in the same environment this notebook is running in and execute the following commands. <br> 
First install [RDKit](https://anaconda.org/conda-forge/rdkit), 
then install [PyTorch](https://pytorch.org/get-started/locally/),
finally install [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html):

>```conda install -c conda-forge rdkit```
>
>```conda install pytorch torchvision torchaudio cpuonly -c pytorch```
>
>```conda install pyg -c pyg```

Import the packages.


In [ ]:
from rdkit import Chem
import torch
import torch_geometric

<a id='loading-existing-datasets'></a>

## Loading Existing Datasets
PyTorch Geometric ships with a lot of graph datasets, a list of which can be found [here](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html). <br>For molecular machine learning, the datasets from the paper ["MoleculeNet, A Benchmark for Molecular Machine Learning"](https://arxiv.org/abs/1703.00564) are the most relevant, although some of the datasets collected by the [TU Dortmund](https://chrsmrrs.github.io/datasets) also include molecular graphs. In addition, a graph dataset from the [ZINC database](https://pubs.acs.org/doi/abs/10.1021/acs.jcim.5b00559) used with 250 000 molecular graphs is available.
<br><br>
As an example, we are going to load the Tox21 dataset availiable as part of the MoleculeNet datasets.

In [ ]:
dataset = torch_geometric.datasets.MoleculeNet(root=".", name="Tox21")

 The data will only be downloaded once and saved in a new folder in the directory of this notebook. We will explore how this works later. <br> <br> 
 We can now inspect the data we downloaded:

In [ ]:
print(dataset)
print(dataset[0])

We can see that the dataset encompasses 7831 molecular graphs. If we look at the first graph in the dataset, PyTorch Geometric shows us a `Data` object. This object is the graph in a computer readable format and works much like a python dictionary. We can access each attribute individually.  


In [ ]:
dataset[0].x

The node feature matrix `x=[16, 9]` of the molecular graph has the shape `[num_nodes, num_features]`. The first graph has 16 nodes, and because `num_nodes` for molecular graphs is normally the number of atoms, we can see that the first molecule has 16 atoms. In this version of the dataset, each atom has a 9-dimensional feature vector, containing atomic number and chirality, as well as other additional atom features such as formal charge and whether the atom is in the ring or not, which was added from the [Open Graph Benchmark](https://ogb.stanford.edu/docs/graphprop/).

In [ ]:
dataset[0].edge_index

The `edge_index` denotes the connectivity of the nodes and is in molecular graphs a representation of the bonds. It has the shape `[2, num_edges]`, showing the direction of edges. Because bonds have no direction, each node pair appears twice. Saving the connections this way is a more memory efficient way than using an adjacency matrix. <br><br>
Node feature matrix and edge index are enough to define our graph. But we also need labels to actually train on our data.

In [ ]:
dataset[0].y

The labels are saved in `y` and for the Tox21 dataset, they denote the 12 different toxicological tests for each compound (0 means inactive and 1 means active). <br><br>
The `edge_attr` attribute is an edge feature matrix, which for molecular graphs normally encodes the bond order. Many message passing algorithms used to train graph neural networks do not use edge attributes, so we will ignore them for now. <br><br>
Important is the `smiles` label each graph has. This allows us to look at the molecule and even make changes to it with RDKit. Unfortunately not every molecular graph dataset in PyTorch Geometric comes with SMILES attached.

In [ ]:
Chem.Draw.MolToImage(Chem.MolFromSmiles(dataset[0].smiles))

<a id='creating-your-own-datasets'></a>

## Creating Your Own Datasets
Often we want to try out different feature sets or work on a custom dataset that is not implemented in PyTorch Geometric. While it is possible to just create a (pickled) `list` of PyTorch Geometric `Data` objects and feed the list to a graph neural network, it is more efficient to create a custom dataset. How to create a dataset is docomented [here](https://pytorch-geometric.readthedocs.io/en/latest/notes/create_dataset.html) and also describes how to create datasets that do not fit into memory. <br><br>
Assuming we have molecular data in the form of SMILES and some arbitrary label, saved as a `.csv`:

In [ ]:
import pandas as pd
example_data = pd.read_csv('data.csv')
example_data

We need to create a class that inherits from `InMemoryDataset` (or from `Dataset` for large datasets). In the class four methods need to be implemented for the dataset to work.
* `raw_file_names()` lists files in the `raw_dir` which have to be found in order to skip the download.
* `processed_file_names()`lists files in the `processed_dir` which have to be found in order to skip the processing.
* `download()` downloads the raw data into the `raw_dir`.
* `process()` processes raw data and saves it to the `processed_dir`.

The additional methods `transform()`, `pre_transform()` and `pre_filter()` will not be implemented in this example but can be read about [here](https://pytorch-geometric.readthedocs.io/en/latest/notes/create_dataset.html).

>We start by initiating the class, and add an additional parameter `create_edge_attr`.
```py
class MyDataset(torch_geometric.data.InMemoryDataset):
    def __init__(self, root, create_edge_attr=False):
        self.create_edge_attr=create_edge_attr
        super().__init__(root)
        
        self.data, self.slices = torch.load(self.processed_paths[0])
```
>In the `raw_file_names()` method we return the name of our raw data.
```py
    @property
    def raw_file_names(self):
        return ['data.csv']
```
>In the `processed_file_names()` method we return the name we want our processed data to have.
```py
    @property
    def processed_file_names(self):
        return ['processed_data.pt']
```
>We repurpose the `download()` method to simply copy our original `.csv` file to the raw folder. PyTorch Geometric allso has functions like[`download_url`](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html) to directly download data from the internet. This method will only be called, if the previously specified files are not in the raw directory.
```py
    def download(self):
        import shutil
        shutil.copy('data.csv', self.raw_dir+'/data.csv')
```

>In the `process()` method the graphs are actually created from the smiles we supplied. While PyTorch Geometric has a [`from_smiles`](https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/utils/smiles.html#from_smiles) function defined, here we will learn how we could featurize the molecules in different ways. This method will only be called if the files in the processed directory are not present. So you will have to **manually delete the processed files** if you made changes to the `process()` method after calling it for the first time.<br><br>We start by reading in the raw data and create an list to save our graphs in. 
```py
    def process(self):
        df = pd.read_csv(self.raw_paths[0])
        data_list = [None]*len(df)
```
>We now iterate over our raw data and create `mols`from the smiles.
```py
        for idx, row in df.iterrows():
            smiles = row['smiles']
            label = row['labels']
            mol = Chem.MolFromSmiles(smiles)
```
>We create an empty node feature matrix, and to fill it, we iterate over the atoms and add the atomic features. Here we only use 5 features, but this can of course be expanded.
```py
            x = torch.empty([mol.GetNumAtoms(), 5], dtype=torch.long)
            for i, atom in enumerate(mol.GetAtoms()):
                x[i][0] = atom.GetAtomicNum()
                x[i][1] = atom.GetDegree()
                x[i][2] = atom.GetFormalCharge()
                x[i][3] = atom.GetHybridization()
                x[i][4] = atom.GetIsAromatic()
```
>To create the edge_index, we need to know wether we need edge attributes. RDKit supplies the adjacency matrix, which is easily transformed to the sparse `edge_index`. If, however, we want to use edge attributes as well, we have to iterate over the bonds as we did with the atom features. We also need to supply each bond twice, because molecular graphs are undirected wich leads to each edge appearing twice. We will simply use the RDKit bond type as our edge attributes. 
```py
            if self.create_edge_attr:
                edge_indices = []
                edge_attrs = []
                for i, bond in enumerate(mol.GetBonds()):
                    atom1 = bond.GetBeginAtomIdx()
                    atom2 = bond.GetEndAtomIdx()
                    
                    # cast aromatic value (1.5) to int 4
                    attr = 4 if bond.GetIsAromatic() else int(bond.GetBondTypeAsDouble()) 
                    
                    edge_indices += [[atom1, atom2], [atom2,atom1]]
                    edge_attrs += [attr, attr]
                edge_index = torch.tensor(edge_indices)
                edge_index = edge_index.t().to(torch.long).view(2,-1)
                edge_attr = torch.tensor(edge_attrs, dtype=torch.long).unsqueeze(1)

                if edge_index.numel() > 0:  # Sort indices.
                    perm = (edge_index[0] * x.size(0) + edge_index[1]).argsort()
                    edge_index, edge_attr = edge_index[:, perm], edge_attr[perm]
            else:
                adj = Chem.rdmolops.GetAdjacencyMatrix(mol)
                edge_index, _ = torch_geometric.utils.dense_to_sparse(torch.tensor(adj))
```
>Now we only need to cast the label to a tensor as well and then create the `Data` object from everything. It is important to also add the SMILES to the graph, if we later want to look at the molecule.
```py
            label = torch.tensor([[label]], dtype=torch.long)
            
            data_list[idx] = torch_geometric.data.Data(x=x, 
                                                       edge_index=edge_index, 
                                                       y=label,
                                                       smiles=smiles)
            if self.create_edge_attr: data_list[idx].edge_attr = edge_attr
```
>Finally, all of our created graphs are saved in a more efficient way. The last line of the constructor will load and unpack this data.
```py
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])
```

>Now we can create and inspect our dataset. (You can also just copy the next cell and make minor modification to create your own dataset.)

In [ ]:
class MyDataset(torch_geometric.data.InMemoryDataset):
    def __init__(self, root, create_edge_attr=False):
        self.create_edge_attr=create_edge_attr
        super().__init__(root)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def raw_file_names(self):
        return ['data.csv']

    @property
    def processed_file_names(self):
        return ['processed_data.pt']

    def download(self):
        import shutil
        shutil.copy('data.csv', self.raw_dir+'/data.csv')

    def process(self):
        df = pd.read_csv(self.raw_paths[0])
        data_list = [None]*len(df)
        for idx, row in df.iterrows():
            smiles = row['smiles']
            label = row['labels']
            mol = Chem.MolFromSmiles(smiles)
            
            x = torch.empty([mol.GetNumAtoms(), 5], dtype=torch.long)
            for i, atom in enumerate(mol.GetAtoms()):
                x[i][0] = atom.GetAtomicNum()
                x[i][1] = atom.GetDegree()
                x[i][2] = atom.GetFormalCharge()
                x[i][3] = atom.GetHybridization()
                x[i][4] = atom.GetIsAromatic()
            
            if self.create_edge_attr:
                edge_indices = []
                edge_attrs = []
                for i, bond in enumerate(mol.GetBonds()):
                    atom1 = bond.GetBeginAtomIdx()
                    atom2 = bond.GetEndAtomIdx()
                    attr = 4 if bond.GetIsAromatic() else int(bond.GetBondTypeAsDouble()) # cast aromatic value (1.5) to int 4                   
                    edge_indices += [[atom1, atom2], [atom2,atom1]]
                    edge_attrs += [attr, attr]
                edge_index = torch.tensor(edge_indices)
                edge_index = edge_index.t().to(torch.long).view(2,-1)
                edge_attr = torch.tensor(edge_attrs, dtype=torch.long).unsqueeze(1)

                if edge_index.numel() > 0:  # Sort indices.
                    perm = (edge_index[0] * x.size(0) + edge_index[1]).argsort()
                    edge_index, edge_attr = edge_index[:, perm], edge_attr[perm]
            else:
                adj = Chem.rdmolops.GetAdjacencyMatrix(mol)
                edge_index, _ = torch_geometric.utils.dense_to_sparse(torch.tensor(adj))
            
            label = torch.tensor([[label]], dtype=torch.long)
            
            data_list[idx] = torch_geometric.data.Data(x=x, 
                                                  edge_index=edge_index, 
                                                  y=label,
                                                  smiles=smiles)
            if self.create_edge_attr: data_list[idx].edge_attr = edge_attr

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])
        
my_set = MyDataset('tox21', create_edge_attr = True)

print(my_set)
print(my_set[1])

In [ ]:
print(my_set[1].x)
print(my_set[1].edge_index)
print(my_set[1].y)
print(my_set[1].smiles)
print(my_set[1].edge_attr)

In the next section we will learn how to train a simple graph neural network to on the Tox21 dataset.

<a id='training-a-simple-model'></a>

## Training a Simple Model

Graph neural networks use message passing algorithms to pass information along the edges of a graph. The popular algorithms are already implemented as convolutional layers in PyTorch Geometric, a list of which can be found [here](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html). Classes like `Sequential` or `BatchNorm` from PyTorch have also been implemented for PyTorch Geometric.<br><br>If you already know about training neural networks, you can probably skip this section.<br><br>We will build our model as a class that inherits from PyTorchs `Module` class. We start with two Graph Attention Convolutional Layers, followed by a max-pooling layer and fully connected layer, using ReLUs as nonlinear activation functions.

In [ ]:
import torch
from torch_geometric.nn import GATConv, global_max_pool
from torch.nn import functional as F
from torch.nn import Linear

class MyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(9, 200)
        self.conv2 = GATConv(200, 200)
        self.fc1 = Linear(200, 12)

    def forward(self, x, edge_index, batch):
       
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = global_max_pool(x, batch)
        x = self.fc1(x)
        
        return x


We will train our model on the Tox21 Dataset that was downloaded earlier. For the convolutional layers it is neccessary to cast the node features to float tensors and we randomly split our dataset into a train- and testset. (Unfortunatly we have to change the nice dataset to a list if we want to actually change the datatype of the tensors.)<br>We also make use of the `loader` functionality to train the data in batches. 

In [ ]:
import torch_geometric
from torch_geometric import loader

dataset = torch_geometric.datasets.MoleculeNet(root=".", name="Tox21")
dataset = dataset.shuffle()
dataset = list(dataset)
for graph in dataset:
    graph.x = graph.x.type(torch.float32)
    graph.y = torch.nan_to_num(graph.y)

train_set = dataset[:int(len(dataset)*.8)]
test_set = dataset[int(len(dataset)*.8):]
train_loader = loader.DataLoader(train_set, batch_size=32)
test_loader = loader.DataLoader(test_set, batch_size=32)

Next, we initialize our model and chose a loss function and an optimizer.

In [ ]:
from torch import optim, nn

device = ('cuda:0' if torch.cuda.is_available() else 'cpu')
model = MyModel().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

Finally, we can train our model on the trainingsset and evaluate it on the test set.

In [ ]:
from sklearn.metrics import roc_auc_score

EPOCHS = 30
for epoch in range(1, EPOCHS+1):
    loss_list = []
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        batch = batch.to(device)
        out = model(batch.x, batch.edge_index, batch.batch)
        loss = criterion(out, batch.y)
        loss_list.append(loss.item())
        loss.backward()
        optimizer.step()

    accuracy_list = []
    roc_auc_list = []
    model.eval()
    for batch in test_loader:
        batch = batch.to(device)
        out = torch.round(torch.sigmoid(model(batch.x, batch.edge_index, batch.batch))).to('cpu').detach()
        batch = batch.to('cpu')
        accuracy = torch.sum(torch.round(torch.sigmoid(out)) == batch.y.cpu())/(batch.y.shape[0]*batch.y.shape[1])
        accuracy_list.append(accuracy)
    print(f'Loss for epoch {epoch} = {sum(loss_list)/len(train_loader):.4f}    Accuracy = {sum(accuracy_list)/len(test_loader):.4f}')
    

Because the dataset is heavily imbalanced, and a high accuracy is reached if the model just sets every value to zero, we also calculate the area under the reciever-operator characteristic for each of the 12 classes.

In [ ]:
from sklearn.metrics import roc_auc_score

label = []
pred = []

model.eval()
for batch in train_loader:
    batch = batch.to(device)
    out = model(batch.x, batch.edge_index, batch.batch)
    label.append(batch.y)
    pred.append(out.detach())

label = torch.cat(label, 0).cpu()
pred = torch.cat(pred, 0).cpu()

roc_auc = [roc_auc_score(label[:, i], pred[:, i]) for i in range(12)]
print([round(x, 4) for x in roc_auc])